# 1. Import Depedencies

In [15]:
# Import GYM Stuff
import gym
from gym import Env
from gym.spaces import Discrete, Box, Tuple, Dict, MultiBinary, MultiDiscrete

# Import Helpers
import os
import random
import numpy as np

# Import Stable Baseline Stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces Note

In [5]:
Discrete(3).sample()

0

In [6]:
Box(0,1, shape=(3,3)).sample()

array([[0.45989925, 0.69893366, 0.38001496],
       [0.28955323, 0.4063222 , 0.78657186],
       [0.781508  , 0.11170615, 0.46415055]], dtype=float32)

In [14]:
Tuple((Discrete(3), Box(0,1, shape=(3,3)))).sample()

(2,
 array([[0.07289439, 0.37878126, 0.58525443],
        [0.5394716 , 0.43762168, 0.4295191 ],
        [0.32884955, 0.07323059, 0.66752845]], dtype=float32))

In [20]:
Dict({'height': Discrete(2), 'speed':Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([39.0432], dtype=float32))])

In [21]:
MultiBinary(4).sample()

array([0, 0, 1, 0], dtype=int8)

In [29]:
MultiDiscrete([5,2,2]).sample()

array([4, 0, 1])

# 3. Building an Environment
- Build an agent to give us the best shower possible
- Randomly temperature
- 37 and 39 degrees its optimum temperatur

In [38]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set Start Temperature
        self.state = 38 + random.randint(-3, 3)
        # Set Shower Length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        
        # Apply temp adj
        self.state += action -1
        
        # Reduce shower length by 1 second (Decrease shower time)
        self.shower_length -= 1
        
        # Calculate Reward
        if self.state >=37 and self.state <=39:
            reward = 1
        else:
            reward = -1
        
        # Check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info
        
    def render(self):
        # Implement Visualization
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [39]:
env = ShowerEnv()

/home/brianadit24/Downloads/AI_Research/rlcourse_env/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [40]:
env.observation_space.sample()

array([34.600204], dtype=float32)

In [41]:
env.action_space.sample()

2

In [74]:
env.reset()

array([35.])

In [79]:
from stable_baselines3.common.env_checker import check_env
check_env(env, warn=True)

# 4. Test Environment

In [81]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score+=reward
    print(f"Episode: {episode} Score: {score}")
env.close()

Episode: 1 Score: 10
Episode: 2 Score: -2
Episode: 3 Score: 30
Episode: 4 Score: -58
Episode: 5 Score: -60


# 5. Train Model

In [82]:
log_path = os.path.join('Training', 'Logs')

In [83]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [84]:
model.learn(total_timesteps=400000)

Logging to Training/Logs/PPO_9
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -30.2    |
| time/              |          |
|    fps             | 449      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -31.9       |
| time/                   |             |
|    fps                  | 340         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015733294 |
|    clip_fraction        | 0.115       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.000629    |

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -27         |
| time/                   |             |
|    fps                  | 334         |
|    iterations           | 11          |
|    time_elapsed         | 67          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.000975844 |
|    clip_fraction        | 0.0175      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 6.56e-07    |
|    learning_rate        | 0.0003      |
|    loss                 | 35          |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.00129    |
|    value_loss           | 73.6        |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60      

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 0.84        |
| time/                   |             |
|    fps                  | 360         |
|    iterations           | 21          |
|    time_elapsed         | 119         |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.010840302 |
|    clip_fraction        | 0.159       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.04       |
|    explained_variance   | -0.00361    |
|    learning_rate        | 0.0003      |
|    loss                 | 15.4        |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.0087     |
|    value_loss           | 33.8        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 39          |
| time/                   |             |
|    fps                  | 374         |
|    iterations           | 31          |
|    time_elapsed         | 169         |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.002644265 |
|    clip_fraction        | 0.174       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.891      |
|    explained_variance   | 1.19e-07    |
|    learning_rate        | 0.0003      |
|    loss                 | 27.9        |
|    n_updates            | 300         |
|    policy_gradient_loss | 0.00361     |
|    value_loss           | 53          |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | 41.1       |
| time/                   |            |
|    fps                  | 378        |
|    iterations           | 41         |
|    time_elapsed         | 221        |
|    total_timesteps      | 83968      |
| train/                  |            |
|    approx_kl            | 0.03506861 |
|    clip_fraction        | 0.28       |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.822     |
|    explained_variance   | -2.49e-05  |
|    learning_rate        | 0.0003     |
|    loss                 | 39.9       |
|    n_updates            | 400        |
|    policy_gradient_loss | 0.0124     |
|    value_loss           | 58.8       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_m

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 56.3        |
| time/                   |             |
|    fps                  | 397         |
|    iterations           | 151         |
|    time_elapsed         | 778         |
|    total_timesteps      | 309248      |
| train/                  |             |
|    approx_kl            | 0.007821042 |
|    clip_fraction        | 0.256       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.895      |
|    explained_variance   | 1.39e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 57.6        |
|    n_updates            | 1500        |
|    policy_gradient_loss | 0.0175      |
|    value_loss           | 111         |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60      

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 56          |
| time/                   |             |
|    fps                  | 393         |
|    iterations           | 161         |
|    time_elapsed         | 837         |
|    total_timesteps      | 329728      |
| train/                  |             |
|    approx_kl            | 0.032060042 |
|    clip_fraction        | 0.277       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.9        |
|    explained_variance   | -1.94e-05   |
|    learning_rate        | 0.0003      |
|    loss                 | 31.9        |
|    n_updates            | 1600        |
|    policy_gradient_loss | 0.0182      |
|    value_loss           | 97.5        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 56.2        |
| time/                   |             |
|    fps                  | 389         |
|    iterations           | 171         |
|    time_elapsed         | 899         |
|    total_timesteps      | 350208      |
| train/                  |             |
|    approx_kl            | 0.013454216 |
|    clip_fraction        | 0.278       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.888      |
|    explained_variance   | 8.94e-07    |
|    learning_rate        | 0.0003      |
|    loss                 | 67.8        |
|    n_updates            | 1700        |
|    policy_gradient_loss | 0.0187      |
|    value_loss           | 103         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | 55.6       |
| time/                   |            |
|    fps                  | 385        |
|    iterations           | 181        |
|    time_elapsed         | 960        |
|    total_timesteps      | 370688     |
| train/                  |            |
|    approx_kl            | 0.07012292 |
|    clip_fraction        | 0.248      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.892     |
|    explained_variance   | 7.27e-06   |
|    learning_rate        | 0.0003     |
|    loss                 | 66.5       |
|    n_updates            | 1800       |
|    policy_gradient_loss | 0.017      |
|    value_loss           | 113        |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 54.7        |
| time/                   |             |
|    fps                  | 382         |
|    iterations           | 191         |
|    time_elapsed         | 1021        |
|    total_timesteps      | 391168      |
| train/                  |             |
|    approx_kl            | 0.005493913 |
|    clip_fraction        | 0.2         |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.837      |
|    explained_variance   | -9.89e-06   |
|    learning_rate        | 0.0003      |
|    loss                 | 53.7        |
|    n_updates            | 1900        |
|    policy_gradient_loss | 0.0144      |
|    value_loss           | 109         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

# 6. Save Model & Evaluate

In [85]:
shower_path = os.path.join('Training', 'SavedModels', 'Shower_CustomEnv_Model_PPO')

In [86]:
model.save(shower_path)

In [87]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/home/brianadit24/Downloads/AI_Research/rlcourse_env/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(59.8, 0.6)